In [296]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

In [297]:
timeframe = 40
stride = 1
num_features = 5

In [298]:
df_x_train = pd.read_csv('./data/X_train_transformed.csv', sep=' ')
df_y_train = pd.read_csv('./data/y_train_transformed.csv', sep=' ')
df_x_test = pd.read_csv('./data/X_test_transformed.csv', sep=' ')
df_y_test = pd.read_csv('./data/y_test_transformed.csv', sep=' ')


In [299]:
x_train = np.loadtxt('./data/X_train_transformed.csv')
x_train = x_train.reshape(x_train.shape[0], timeframe, num_features)
y_train = np.loadtxt('./data/y_train_transformed.csv').astype(int)
x_test = np.loadtxt('./data/X_test_transformed.csv')
x_test = x_test.reshape(x_test.shape[0], timeframe, num_features)
y_test = np.loadtxt('./data/y_test_transformed.csv').astype(int)

In [300]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((21560, 40, 5), (21560,), (5361, 40, 5), (5361,))

In [301]:
n_classes = 3

In [302]:
model = Sequential([
    Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(timeframe, num_features)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(n_classes, activation='softmax')
])

/home/bina/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [303]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [304]:
print(model.summary())

Model: "sequential_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_18 (Conv1D)              │ (None, 39, 64)         │           704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_18 (MaxPooling1D) │ (None, 19, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_18 (Flatten)            │ (None, 1216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 64)             │        77,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 78,787 (307.76 KB)

 Trainable params: 78,787 (307.76 KB)

 Non-trainable params: 0 (0.00 B)

None


In [305]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [306]:
unique_labels = np.unique(y_train)
class_mapping = {label: i for i, label in enumerate(unique_labels)}
y_train_encoded = np.array([class_mapping[label] for label in y_train])
y_test_encoded = np.array([class_mapping[label] for label in y_test])
y_train_one_hot = to_categorical(y_train_encoded, num_classes=len(unique_labels))
y_test_one_hot = to_categorical(y_test_encoded, num_classes=len(unique_labels))

In [308]:
history = model.fit(x_train, y_train_one_hot, epochs=50, batch_size=32, validation_data=(x_test, y_test_one_hot), callbacks=[early_stopping])

Epoch 1/50


674/674 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5862 - loss: 0.8736 - val_accuracy: 0.5917 - val_loss: 0.8599
Epoch 2/50
674/674 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5932 - loss: 0.8609 - val_accuracy: 0.5917 - val_loss: 0.8593
Epoch 3/50
674/674 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5922 - loss: 0.8607 - val_accuracy: 0.5917 - val_loss: 0.8596
Epoch 4/50
674/674 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5934 - loss: 0.8546 - val_accuracy: 0.5917 - val_loss: 0.8594
Epoch 5/50
674/674 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5938 - loss: 0.8579 - val_accuracy: 0.5917 - val_loss: 0.8633
Epoch 6/50
674/674 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5885 - loss: 0.8585 - val_accuracy: 0.5917 - val_loss: 0.8605
Epoch 7/50
674/674 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5893 - loss: 0.8551 - val_accuracy: 0.5917 - val_loss: 0.8670


In [309]:
loss, accuracy = model.evaluate(x_test, y_test_one_hot)
print("Loss:", loss)
print("Accuracy:", accuracy)

168/168 ━━━━━━━━━━━━━━━━━━━━ 0s 667us/step - accuracy: 0.5915 - loss: 0.8658
Loss: 0.8593106865882874
Accuracy: 0.591680645942688
